In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics
% graphics_toolkit ('qt')
warning('off','all');

rand('state',123456)

executing gpml startup script...


In [2]:
ttcs = 200;
knc = 20; % for each data point consider its k nearest clusters
gamma = 0.1;
sig_temp = 0.5;
sig_max = 0.6; sig_min = 0.2;

In [3]:
load ('./data/spatial_network.mat');
[ndata, ~] = size(A);
X = A(:,1:3);
Y = A(:,4);
R = randperm(ndata);        
ori_xvec_test = X(R(1:34874),:);
ori_yvec_test = Y(R(1:34874));
R(1:34874) = [];
ori_xvec = X(R,:);          
ori_yvec = Y(R);
% x = x1(1:45000,:);
% y = y1(1:45000);
% xs = x1(45001:50000,:);
% ys = y1(45001:50000);
n = 400000; nt = 34874;   %Ms = 500

In [4]:
ttm = round(n / (ttcs/2));
% fast_induce_step = 50;
induce_step = 100;

xnorm = 1; ynorm = 1;

In [5]:
% input normalization
if xnorm == 1
    norm_xmean = mean(ori_xvec);
    norm_xstd = std(ori_xvec);
    xvec = (ori_xvec - repmat(norm_xmean,n,1)) ./ repmat(norm_xstd,n,1);    
    xvec_test = (ori_xvec_test - repmat(norm_xmean,nt,1)) ./ repmat(norm_xstd,nt,1);
    xvec_val = (ori_xvec_val - repmat(norm_xmean,nv,1)) ./ repmat(norm_xstd,nv,1);
else
    xvec = ori_xvec;
    xvec_test = ori_xvec_test;
    xvec_val = ori_xvec_val;
end

% output normalization
if ynorm == 1
    norm_fmean = mean(ori_yvec);
    norm_fstd = std(ori_yvec);
    yvec = (ori_yvec - norm_fmean) / norm_fstd;
    yvec_test = (ori_yvec_test - norm_fmean) / norm_fstd;
    yvec_val = (ori_yvec_val - norm_fmean) / norm_fstd;
else
    yvec = ori_yvec;
    yvec_test = ori_yvec_test;
    yvec_test_val = ori_yvec_val;
end

error: 'ori_xvec_val' undefined near line 7 column 17
error: 'ori_yvec_val' undefined near line 7 column 17


In [6]:
testsize = nt; % size of test data
valsize = nv;
induce_step = 100;

error: 'nv' undefined near line 1 column 11


In [7]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(xvec,2);
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = ttm+1;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

meanfunc = [];                    % empty: don't use a mean function
covfunc = opts.covfunc;              % Squared Exponental covariance function
likfunc = opts.likfunc;              % Gaussian likelihood
inffunc = opts.inffunc;
km_iters = 1e4;
opts.induce_step = induce_step ;

In [8]:
% hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];    
opts.numOptFC = 50 ;
opts.Ms = ttm+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
% opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, []};
opts.covfuncF = covfuncF;
opts.compute_hyp = 0;

In [9]:
% default partition
dcs_ecs_r = 0.5;
dcs = round(ttcs*dcs_ecs_r) % size of the communication set
ecs = ttcs - dcs % size of other experts

dcs =  100
ecs =  100


Baselines.

In [10]:
tic();

n_per = dcs ; % size of Dc
m = mn = round(n / ecs); % mn is the number of experts (normal)
Indics = randperm(n) ;
I_com = Indics(1:n_per) ; % randomly select communication set
[idx, C] = kmeans(xvec, mn,'MaxIter',km_iters);    

% hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];    
opts.numOptFC = 30 ;
opts.Ms = mn+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.induce_size = dcs;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
opts.I_com = I_com;
% opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, xvec(I_com,:)};
opts.covfuncF = covfuncF;
opts.compute_hyp = 0;

g_opts = opts;
g_opts.compute_hyp = 1;
g_opts.grbcm_baseline = 1;
g_opts.global_index = ones(n,1);
[g_models, grbcm_llhs] = GRBCM_train(xvec,yvec,idx,g_opts); 
opts.hyp = g_models{1}.hyp;
g_opts.hyp = g_models{1}.hyp;
[tmu,ts2] = GRBCM_predict(xvec_test,g_models, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[grbcmMSE,grbcmSMSE,grbcmMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);

Optimizing hyps in training...
Linesearch     30;  Value -4.066693e+05


In [11]:
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM', n_per, grbcmMSE,grbcmSMSE,grbcmMSLL);
grbcm_llh = grbcm_llhs(end)
g_opts.compute_hyp = 0;

time_grbcm_opt = toc()

GRBCM (Dc size 100): MSE 4.86515719, SMSE 0.01394527, MSLL -2.72878801
grbcm_llh =   -4.0667e+05
time_grbcm_opt =  7196.5


other baselines

In [12]:
criterion = 'RBCM';
[tmu,ts2,t_dGP_predict] = aggregation_predict(xvec_test,g_models,criterion, 1, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[rbcmMSE,rbcmSMSE,rbcmMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', criterion, n_per, rbcmMSE,rbcmSMSE,rbcmMSLL);

criterion = 'BCM';
[tmu,ts2,t_dGP_predict] = aggregation_predict(xvec_test,g_models,criterion, 1, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[bcmMSE,bcmSMSE,bcmMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', criterion, n_per, bcmMSE,bcmSMSE,bcmMSLL);

criterion = 'PoE';
[tmu,ts2,t_dGP_predict] = aggregation_predict(xvec_test,g_models,criterion, 1, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[poeMSE,poeSMSE,poeMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', criterion, n_per, poeMSE,poeSMSE,poeMSLL);

criterion = 'GPoE';
[tmu,ts2,t_dGP_predict] = aggregation_predict(xvec_test,g_models,criterion, 1, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[gpoeMSE,gpoeSMSE,gpoeMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', criterion, n_per, gpoeMSE,gpoeSMSE,gpoeMSLL);

RBCM (Dc size 100): MSE 4.86631359, SMSE 0.0139, MSLL -2.7276
BCM (Dc size 100): MSE 3.02216297, SMSE 0.0087, MSLL -3.0589
PoE (Dc size 100): MSE 305.40723390, SMSE 0.8754, MSLL 3676.2099
GPoE (Dc size 100): MSE    NaN, SMSE    NaN, MSLL    NaN


In [ ]:
tic();
% compute distance matrix
ds = zeros(n, m);
for i=1:n
    for j=1:m
        ds(i,j) = norm(xvec(i, :) - C(j,:));
    end
end

% 1st closest cluster centers
[~, fcc] = min(ds, [], 2);
% % set to inf
% for i=1:n
%     ds(i, fcc(i)) = 1e10;
% end

% consider knc closest cluster centers
[~, ranks] = sort(ds, 2);
mask = zeros(n,m); rk_mat = zeros(n,m); 
rk = zeros(n, 1);

for iik=1:n
    mask(iik, ranks(iik, 1:knc+1)) = 1;
end

for iim=1:m
    if mod(iim, 10)==0
        fprintf('processing distance: %d/%d\n', iim, m);
    end
    xidx = logical(mask(:, iim));
    evalxvec = xvec(xidx, :);
%     size(evalxvec)
    [tmp_mu, tmp_sig2] = gp(g_opts.hyp,inffunc,meanfunc, ...
        covfunc,likfunc,g_models{iim+1}.X,g_models{iim+1}.Y,evalxvec);
    rk_mat(xidx, iim) = log(tmp_sig2);
end

for i=1:n
    rk(i) = sum(rk_mat(i, :)) - 2*rk_mat(i, fcc(i));
end

time_cid = toc()

In [ ]:
mean(rk)

In [ ]:
trk = -30;
sig_temp = 0.1;

th = min(rk)+(max(rk)-min(rk))*gamma;
gi = rk > th;

mrk = mean(rk(gi))

rr=mrk-trk
dcs_ecs_r = sigmoid(rr, sig_temp);
dcs_ecs_r = min(dcs_ecs_r, sig_max);
dcs_ecs_r = max(dcs_ecs_r, sig_min);
dcs_ecs_r
dcs = round(ttcs*dcs_ecs_r)
ecs = ttcs - dcs
m = round(n / ecs)

In [ ]:
tic();
n_per = dcs ; % size of Dc
Indics = randperm(n) ;
I_com = Indics(1:n_per) ; % randomly select communication set
[idx, C] = kmeans(xvec, m,'MaxIter',km_iters);    

opts.compute_hyp = 0;
% hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];    
opts.numOptFC = 30 ;
opts.Ms = m+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.induce_size = dcs;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
opts.I_com = I_com;
% opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, xvec(I_com,:)};
opts.covfuncF = covfuncF;

g_opts = opts;
% g_opts.compute_hyp = 1;
g_opts.grbcm_baseline = 1;
g_opts.global_index = ones(n,1);
g_models = GRBCM_train(xvec,yvec,idx,g_opts); 
time_overhead = toc()

Optimize VFE and SPGP.

In [ ]:
I_com2 = Indics(1:dcs) ; % randomly select communication set

In [ ]:
tic();
% % VFE Baseline
vfe_opts = opts;
vfe_opts.induce_type = 'VFE_opt';
xu = xvec(I_com2, :);
inffunc = @(varargin) infGaussLik(varargin{:}, struct('s', 0.0));
vfe_hyp = opts.hyp;
vfe_hyp.xu = xu;
[vfe_hyp, vfe_llhs] = minimize(vfe_hyp,@sp_gp,-vfe_opts.induce_step,inffunc,meanfunc,covfuncF,likfunc,xvec,yvec);
vfe_opts.hyp = opts.hyp;
vfe_opts.xu = vfe_hyp.xu;
vfe_opts.inffunc = @infGaussLik; vfe_opts.meanfunc = meanfunc; vfe_opts.covfuncF = covfuncF; vfe_opts.likfunc = likfunc;
vfe_opts.covfunc = covfunc;
[tmu, ts2] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, xvec_test);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[vfeMSE,vfeSMSE,vfeMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'VFE baseline', n_per, vfeMSE,vfeSMSE,vfeMSLL);
vfe_llh = vfe_llhs(end)
% vfe0_smse_rec(ki) = vfeSMSE; vfe0_msll_rec(ki) = vfeMSLL;
[yu, su] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, vfe_opts.xu);
vfe_opts.yu = yu; vfe_opts.su = su; 
time_vfe_opt = toc()

In [ ]:
tic();
xu_idx = randperm(size(vfe_opts.xu, 1))(1:dcs);
grbcm_vfe_opts = vfe_opts;
grbcm_vfe_opts.xu = vfe_opts.xu(xu_idx,:);
grbcm_vfe_opts.yu = vfe_opts.yu(xu_idx,:);

grbcm_vfe_opts.global_index = gi;
models = GRBCM_train(xvec,yvec,idx,grbcm_vfe_opts); % use hyp of vfe
[tmu,ts2] = GRBCM_predict(xvec_test,models,grbcm_vfe_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[vfeMSE0,vfeSMSE0,vfeMSLL0] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);

In [25]:
fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', 'GRBCM++ (VFE)', n_per, vfeMSE0,vfeSMSE0,vfeMSLL0);
time_eval_grbcm_vfe = toc()

GRBCM++ (VFE) (Dc size 120): 
MSE 4.90918042, SMSE 0.0141, MSLL -2.7402
time_eval_grbcm_vfe =    1.9557e+04


In [ ]:
tic();
sp_opts = opts;
sp_opts.induce_size = dcs;
sp_opts.induce_type = 'SPGP_opt';
hyp_init(1:d,1) = -2*opts.hyp.cov(1:d);
hyp_init(d+1,1) = 2*opts.hyp.cov(d+1);
hyp_init(d+2,1) = 2*opts.hyp.lik;

xu = xvec(I_com2, :);
w_init = [reshape(xu,sp_opts.induce_size*d,1);hyp_init];
[w,spgp_llhs] = minimize(w_init,'spgp_lik_nohyp',-sp_opts.induce_step,yvec,xvec,sp_opts.induce_size);
xb = reshape(w(1:sp_opts.induce_size*d,1),sp_opts.induce_size,d);
sp_opts.xu = xb;
sp_opts.sp_hyp = w(sp_opts.induce_size*d+1:end,1);
sp_opts.hyp = opts.hyp;
[tmu,ts2] = spgp_pred(sp_opts.yvec,sp_opts.xvec,sp_opts.xu,xvec_test,sp_opts.sp_hyp);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[spgpMSE,spgpSMSE,spgpMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'SPSG baseline', n_per, spgpMSE,spgpSMSE,spgpMSLL);
spgp_llh = spgp_llhs(end)
% spgp0_smse_rec(ki) = spgpSMSE; spgp0_msll_rec(ki) = spgpMSLL;
[yu,su] = spgp_pred(sp_opts.yvec,sp_opts.xvec,sp_opts.xu,sp_opts.xu,sp_opts.sp_hyp);
sp_opts.yu = yu; sp_opts.su = su;
time_spgp_opt = toc

In [ ]:
tic();
xu_idx = randperm(size(sp_opts.xu, 1))(1:dcs);
grbcm_spgp_opts = sp_opts;
grbcm_spgp_opts.xu = sp_opts.xu(xu_idx,:);
grbcm_spgp_opts.yu = sp_opts.yu(xu_idx,:);

grbcm_spgp_opts.global_index = gi;
models = GRBCM_train(xvec,yvec,idx,grbcm_spgp_opts); % use hyp of vfe
[tmu,ts2] = GRBCM_predict(xvec_test,models,grbcm_spgp_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[spgpMSE0,spgpSMSE0,spgpMSLL0] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);

In [26]:
fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', 'GRBCM++ (VFE)', n_per, spgpMSE0,spgpSMSE0,spgpMSLL0);
time_eval_grbcm_spgp = toc()

GRBCM++ (VFE) (Dc size 120): 
MSE 4.90853323, SMSE 0.0141, MSLL -2.7435
time_eval_grbcm_spgp =    1.9564e+04


In [ ]:
time_all_grbcm_vfe = (time_grbcm_opt + time_cid + time_vfe_opt + time_overhead + time_eval_grbcm_vfe)/60
time_all_grbcm_spgp = (time_grbcm_opt + time_cid + time_spgp_opt + time_overhead + time_eval_grbcm_spgp)/60
time_all = (time_grbcm_opt + time_cid + time_vfe_opt + time_spgp_opt + time_overhead + time_eval_grbcm_vfe + time_eval_grbcm_spgp)/60